In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
dataset_train = pd.read_csv("../input/gooogle-stock-price/Google_Stock_Price_Train.csv")

In [ ]:
dataset_train.head()

In [ ]:
training_set = dataset_train.iloc[:,1:2].values
training_set

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
plt.plot(training_set_scaled)
plt.show()

In [ ]:
# Creating a data structure with 100 timesteps and 1 output
X_train = []
y_train = []
t = 100  # timesteps
l = len(training_set_scaled)  # l = 1258 in this case
for i in range(t,l):
  X_train.append(training_set_scaled[i-t:i, 0])
  y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape

In [ ]:
#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_train.shape

In [ ]:
y_train

**CREATING LSTM MODEL**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Initialising the RNN
regressor = Sequential()

# First LSTM Layer
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1))) 
regressor.add(Dropout(0.2)) # 
# Second LSTM Layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Third LSTM Layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Fourth LSTM Layer
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

**PREDICTIONS**

In [ ]:
dataset_test = pd.read_csv("../input/gooogle-stock-price/Google_Stock_Price_Test.csv")
real_stock_price = dataset_test.iloc[:,1:2].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0) 
inputs = dataset_total[len(dataset_train)- t:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) 

In [ ]:
# Creating a data structure with 100 timesteps and 1 output
X_test = []
m = len(inputs)   # m = 120 in this case
for i in range(t,m):
  X_test.append(inputs[i-t:i, 0])
X_test  = np.array(X_test)
# Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))  # 1 indicates the number of indicators
# Prediction
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.show()